In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 314 rows, 314 columns and 314 nonzeros
Model has 49455 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-03, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 314 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 314 columns, 0 nonzeros
Presolved model has 49455 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.55087626e+07  0.00000000e+00  2.87e+04 2.51e+02  9.27e+05     0s
   1   1.23843986e+07 -1.62055166e+07  3.48e+03 2.27e-13  2.08e+05     0s
   2   4.11170803e+06 -5.26791800e+06  3.48e-03 1.14e-13  2.99e+04     0s
   3   4.23

In [2]:
A = GLS_Apr_weekday_PM.A;
P = GLS_Apr_weekday_PM.P;
Q = GLS_Apr_weekday_PM.Q;
b = GLS_Apr_weekday_PM.b;
K = GLS_Apr_weekday_PM.K;

In [3]:
size(P, 1), size(P, 2), size(A, 1), size(A, 2) 

(56,314,24,314)

In [4]:
using JuMP

mGLSJulia = Model()

@defVar(mGLSJulia, xi[1:size(P,2)])

@defVar(mGLSJulia, lam[1:size(P,1)] >= 0)

@defVar(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @addConstraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @addNLConstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for i = 1:size(P,2)
    @addNLConstraint(mGLSJulia, sum{p[j,i] * lam[j], j = 1:size(P,1)} >= xi[i])
end

for i = 1:size(P,2)
    @addNLConstraint(mGLSJulia, sum{p[j,i] * lam[j], j = 1:size(P,1)} <= xi[i])
end

    
@setNLObjective(mGLSJulia, Min, (K / 2.0) * sum{xi[i] * Q[i,j] * xi[j], i = 1:size(P,2), j = 1:size(P,2)} - sum{b[i] * xi[i], i = 1:size(P,2)}) 

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    34854
Number of nonzeros in inequality constraint Jacobian.:    70964
Number of nonzeros in Lagrangian Hessian.............:   155587

Total number of variables............................:    17954
                     variables with only lower bounds:    17640
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [5]:
getValue(lam)

56-element Array{Float64,1}:
 137.65 
 131.877
 428.527
 519.445
 576.56 
 425.068
 271.47 
 209.163
 149.371
 448.533
 556.485
 625.715
 462.355
   ⋮    
 346.282
 606.925
 424.97 
 508.912
 245.035
 368.575
 349.753
 332.535
 392.298
 495.415
 601.759
 235.133

In [6]:
getValue(xi)

314-element Array{Float64,1}:
  31.8717
  13.1767
  92.6015
  11.2329
  23.7886
  96.8556
  47.7263
 214.302 
  20.4711
  38.8208
 107.208 
  76.0973
 174.225 
   ⋮     
  55.4596
 263.803 
 123.955 
  12.1134
  91.2639
  69.6957
  20.4514
  45.509 
 238.771 
  16.4598
  23.0087
 195.665 

In [7]:
GLS_Apr_weekday_PM.xi_list

314-element Array{Any,1}:
 381.5   
  62.0596
  53.7844
 143.728 
 147.803 
  35.1334
 175.442 
  84.0441
  49.2077
  78.6722
  38.0729
 132.453 
  62.2551
   ⋮     
  91.3238
  93.3765
  41.1466
  34.2938
  35.9012
  30.5337
 215.068 
 106.124 
 111.63  
 222.638 
  31.7786
  31.1932

In [9]:
getObjectiveValue(mGLSJulia)

-91041.41114630787

In [5]:
getValue(lam)

56-element Array{Float64,1}:
  364.031
  325.364
  387.088
  543.744
  531.093
  533.457
  187.046
  511.458
  326.3  
  459.388
  581.073
  663.681
  610.212
    ⋮    
  714.337
  517.884
  849.503
 1621.38 
  188.789
  239.612
  234.682
  514.833
  383.71 
  402.015
  566.538
  282.776

In [6]:
getObjectiveValue(mGLSJulia)

1.0000000000000073

In [7]:
GLS_Apr_weekday_PM.saveDemandVec(getValue(lam))